In [3]:
import pandas as pd
#import requests
import json
#import yaml
import glob

from myTwitterLibrary import FileImport
from myTwitterLibrary import nlp

pd.set_option('display.max_colwidth', None)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import glob
DataName="chatGPT"


dataList=glob.glob("*"+DataName+"*"+"_data.json")
dataList
print("Number of Json Files: ", len(dataList))
x=FileImport.load_data(dataList[0])

#example Data

x["data"][16]

Number of Json Files:  2


{'public_metrics': {'retweet_count': 0,
  'reply_count': 0,
  'like_count': 2,
  'quote_count': 0,
  'impression_count': 205},
 'created_at': '2023-03-29T21:22:17.000Z',
 'lang': 'en',
 'edit_history_tweet_ids': ['1641189056511119360'],
 'entities': {'hashtags': [{'start': 93, 'end': 101, 'tag': 'ChatGPT'},
   {'start': 102, 'end': 105, 'tag': 'AI'},
   {'start': 106, 'end': 113, 'tag': 'Boston'},
   {'start': 114, 'end': 128, 'tag': 'journorequest'}]},
 'id': '1641189056511119360',
 'text': 'Looking for speaking to Boston students talking about the experience of playing with Chatgpt #ChatGPT #AI #Boston #journorequest',
 'conversation_id': '1641189056511119360',
 'author_id': '1050835728232587264'}

In [47]:
dfk.clean_text

3                                       My latest blog about the support that offers students with
4        years ago, how has it changed? The Growing Need for Skills in Artificial Intelligence via
5    All my time ranting about law schools not teaching lawyering has been for naught. Thanks, . /
Name: clean_text, dtype: object

In [52]:
text=dfk.clean_text[5]
nlp=spacy.load('en_core_web_sm')
nlp.add_pipe("spacytextblob")
doc = nlp(text)

In [54]:
print(doc._.blob.polarity)

0.2


In [57]:
dfk=df.iloc[3:6,:]

dfk.clean_text

import spacy


from spacytextblob.spacytextblob import SpacyTextBlob

def Sentiment(text,nlp):
    
    nlp=spacy.load('en_core_web_sm')
    nlp.add_pipe('spacytextblob')
    doc = nlp(text)
    print(doc)
    polarity=doc._.blob.polarity
    subjectivity=doc._.blob.subjectivity
    return pd.Series([polarity, subjectivity])




dfk.clean_text.apply(Sentiment, nlp=spacy.load('en_core_web_sm'))

My latest blog about the support that offers students with
years ago, how has it changed? The Growing Need for Skills in Artificial Intelligence via
All my time ranting about law schools not teaching lawyering has been for naught. Thanks, . /


,0,1
3,0.3,0.45
4,-0.6,1.00
5,0.2,0.20


In [59]:
from myTwitterLibrary import FileImport
from myTwitterLibrary import nlp

%load_ext autoreload
%autoreload 2

DataName="ChatGpt"

combined_results,combined_media,combined_user,combined_extTweets=FileImport.FileImport(DataName)
df=FileImport.prepare_df(combined_results,combined_media,combined_user, Simplify=True)
df=nlp.NLP_Pipeline(df, sentiment=True, language="en")

df.to_json(FileImport.Today+DataName+"_all.json")
df.columns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2023-03-30_M_ChatGPT__0_data.json, 2023-03-30_M_ChatGPT__1_data.json
Tweets: 1000, Media:206, Users: 1164
cleaning done.
language detection done.
pure english text done. Next: Token & Lemmatizing.
Token & Lemmatizing done. Next: Remove Stopwords.
Stopwording done. Next: sentiment.


Index(['text', 'text_clean', 'created_at', 'id', 'author_id', 'retweet_count',
       'reply_count', 'like_count', 'quote_count', 'impression_count',
       'word_count', 'attachments', 'urls', 'hashtags', 'mentions',
       'media_url', 'username', 'name', 'description', 'location', 'RT',
       'clean_text', 'letters_count', 'language', 'pure_text', 'Lemmata',
       'NoStopwords', 'polarity', 'subjectivity'],
      dtype='object')

# Only add the Sentiment NLP to existing Json Files

In [9]:

from datetime import datetime

#Filenames to rework

Files=[]#['2023-01-22GPT_EDU.json',"2023-01-25nanoMedicine.json","2023-01-25sustain.json"]

for file in Files:
    
    now = datetime.now()

    current_time = now.strftime("%H:%M:%S")
    print("Current Time =", current_time)
    
    print("starting with: ", file)
    #df=pd.read_json(file)
    df[["polarity","subjectivity"]]=df.clean_text.apply(nlp.Sentiment)
    print("writing: ", file)
    df.to_json(FileImport.Today+"_"+file+".json")
    print(file,"done")